In [1]:
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, LayerNormalization
import numpy as np
# from keras.utils import np_utils
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [33]:
not_serve_path = '/content/all_pose_nor_not_serve_real_32_v2.npy'
serve_path = '/content/all_pose_nor_serve_real_32_v2.npy'
ohe = OneHotEncoder()


# not_serve_data = np.load(not_serve_path)
# serve_data = np.load(serve_path)

# all_data = np.concatenate((not_serve_data, serve_data))


In [34]:
# not_serve_label = np.zeros((not_serve_data.shape[0]), dtype="int").reshape(-1,1)
# serve_label = np.ones((serve_data.shape[0]), dtype="int").reshape(-1,1)

In [35]:
# labels = np.vstack([not_serve_label, serve_label])

# y= ohe.fit_transform(labels).toarray().astype(int)


In [36]:
# X_train, X_test, y_train, y_test = train_test_split(all_data,y, test_size = 0.20, shuffle= True)

# print(X_train.shape)
# print(y_train.shape)

# print(X_test.shape)
# print(y_test.shape)

In [37]:
not_serve_data = np.load(not_serve_path)
not_serve_label = np.zeros((not_serve_data.shape[0]), dtype="int").reshape(-1,1)

X_train_not_serve, X_test_not_serve, y_train_not_serve, y_test_not_serve = train_test_split(not_serve_data,not_serve_label, test_size = 0.20, shuffle= True)

print(X_train_not_serve.shape)
print(X_test_not_serve.shape)

(254, 32, 51)
(64, 32, 51)


In [38]:
serve_data = np.load(serve_path)
serve_label = np.ones((serve_data.shape[0]), dtype="int").reshape(-1,1)

X_train_serve, X_test_serve, y_train_serve, y_test_serve = train_test_split(serve_data,serve_label, test_size = 0.20, shuffle= True)

print(X_train_serve.shape)
print(X_test_serve.shape)

(298, 32, 51)
(75, 32, 51)


In [39]:
X_train = np.concatenate((X_train_not_serve, X_train_serve))

X_test = np.concatenate((X_test_not_serve, X_test_serve))

y_train = np.vstack([y_train_not_serve, y_train_serve])
y_test = np.vstack([y_test_not_serve, y_test_serve])

y_train= ohe.fit_transform(y_train).toarray().astype(int)
y_test= ohe.fit_transform(y_test).toarray().astype(int)

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)


(552, 32, 51)
(552, 2)
(139, 32, 51)
(139, 2)


In [41]:
import tensorflow as tf
from tensorflow.keras import layers, models


@tf.keras.utils.register_keras_serializable()
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.4):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)



# Assuming your input shape is (24, 51)
input_shape = (32, 51)
embed_dim = 51  # Adjust based on your needs
num_heads = 4
ff_dim =32
num_transformer_blocks = 4
mlp_dim = 32
num_classes = 2  # Adjust based on the number of activity classes

inputs = tf.keras.Input(shape=input_shape)
# x = layers.Flatten()(inputs)  # Flatten the input to fit into the Transformer model
x = inputs
for i in range(num_transformer_blocks):
    # print(i)
    x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(mlp_dim, activation="relu")(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(num_classes, activation="softmax")(x)

model = tf.keras.Model(inputs=inputs, outputs=x)

# # Compile the model
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])


# Define learning rate decay schedule
initial_learning_rate = 1e-4
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=1000, decay_rate=0.9, staircase=True
)

# Compile the model with the optimizer and learning rate decay
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='categorical_crossentropy',
              metrics=['accuracy'])




In [42]:
# Print the model summary
# model.summary()

# Train the model with your dataset
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


from tensorflow.keras.callbacks import ModelCheckpoint
itertion +=1

# Specify the filepath for saving the best model
checkpoint_filepath = f'lstm_model_32_s2_v1_{itertion}.keras'

# Create ModelCheckpoint callback
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_accuracy',  # You can change this to 'val_loss' or another metric
    mode='max',  # 'max' for accuracy, 'min' for loss
    verbose=1
)

# Assuming X_train, y_train, X_test, y_test are your training and testing data
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=200,
    shuffle=True,
    batch_size=32,
    callbacks=[model_checkpoint]
)

Epoch 1/200
18/18 [==============================] - ETA: 0s - loss: 0.6991 - accuracy: 0.5562
Epoch 1: val_accuracy improved from -inf to 0.60432, saving model to lstm_model_32_s2_v1_5.keras
18/18 [==============================] - 13s 190ms/step - loss: 0.6991 - accuracy: 0.5562 - val_loss: 0.6808 - val_accuracy: 0.6043
Epoch 2/200
17/18 [===========================>..] - ETA: 0s - loss: 0.6574 - accuracy: 0.5993
Epoch 2: val_accuracy improved from 0.60432 to 0.61871, saving model to lstm_model_32_s2_v1_5.keras
18/18 [==============================] - 2s 130ms/step - loss: 0.6603 - accuracy: 0.5942 - val_loss: 0.6543 - val_accuracy: 0.6187
Epoch 3/200
17/18 [===========================>..] - ETA: 0s - loss: 0.6385 - accuracy: 0.6489
Epoch 3: val_accuracy improved from 0.61871 to 0.69784, saving model to lstm_model_32_s2_v1_5.keras
18/18 [==============================] - 3s 149ms/step - loss: 0.6402 - accuracy: 0.6467 - val_loss: 0.6419 - val_accuracy: 0.6978
Epoch 4/200
18/18 [=====